In [31]:
import cv2
import joblib
import numpy as np
import pandas as pd
from skimage.feature import greycomatrix, greycoprops


In [32]:
# Load model
nb = joblib.load('../Model/NB_TRY_NEW.pkl')

In [33]:
def preprocess_and_extract_features(image_path):
    # Load image
    image = cv2.imread(image_path)

    # # Crop image to square
    # height, width, channels = image.shape
    # if height > width:
    #     crop_size = width
    #     y = int((height - width) / 2)
    #     x = 0
    # else:
    #     crop_size = height
    #     x = int((width - height) / 2)
    #     y = 0
    # cropped_image = image[y:y+crop_size, x:x+crop_size]

    # # Resize image
    # resized_image = cv2.resize(cropped_image, (1080, 1080))
    
    # # Konversi gambar ke skala abu-abu
    # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # # Thresholding thresholding pada gambar untuk memisahkan objek dari latar belakang
    # _, thresh = cv2.threshold(
    #     gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # # Temukan kontur pada gambar yang telah dithreshold dan dapatkan kontur terbesar yang diasumsikan sebagai objek daun
    # contours, _ = cv2.findContours(
    #     thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # cnt = max(contours, key=cv2.contourArea)

    # # Dapatkan koordinat bounding box (kotak pembatas) dari kontur
    # x, y, w, h = cv2.boundingRect(cnt)

    # # Dapatkan ROI (Region of Interest) menggunakan koordinat bounding box
    # roi = image[y:y+h, x:x+w]

    # # resize ROI hasil segmentasi
    # resized_image = cv2.resize(roi, (720, 720))

    # Convert image to HSV 
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # hsv = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

    # Perhitungan rata rata hsv
    mean_h, mean_s, mean_v = np.mean(hsv, axis=(0, 1))
    
    # Hitung matriks glcm
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = greycomatrix(image_gray, [5], [0],
                        levels=256, symmetric=True, normed=True)

    # Hitung fitur glcm
    correlation = greycoprops(glcm, 'correlation')[0][0]
    homogeneity = greycoprops(glcm, 'homogeneity')[0][0]
    contrast = greycoprops(glcm, 'contrast')[0][0]

    # Create feature array
    X = np.array([mean_h, mean_s, mean_v, correlation, homogeneity, contrast])
    # X = np.array([mean_h, mean_s, mean_v])

    return X

In [34]:
def classify(image_path):
    # Melakukan pre processing dan ekstraksi fitur
    X = preprocess_and_extract_features(image_path)

    # Mengubah bentuk array
    X_reshaped = X.reshape(1, -1)

    # Membuat dataframe menggunakan nama fitur
    feature_names = ['H', 'S', 'V', 'correlation', 'homogeneity', 'contrast']
    # feature_names = ['H', 'S', 'V']
    X_new = pd.DataFrame(data=X_reshaped, columns=feature_names)

    # Predict label
    y_pred = nb.predict(X_new)

    # Print label
    print(f'Label: {y_pred[0]}')

In [35]:
# Contoh
image_path = '../Dataset/with_Background/5_Label/DataSet_Testing/bercak_merah/bercak_merah (2).jpg'
classify(image_path)

Label: 2


C:\Users\makba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\skimage\feature\__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
C:\Users\makba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
C:\Users\makba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_ve